**<h1 align="center"> Pré-processamento Premier League </h1>**
___

In [37]:
import pandas as pd

DIR = ["dataset/2020-2021.csv","dataset/2021-2022.csv"]

lista_de_dataframes = []

for arquivo_csv in DIR:
    try:
        df_temp = pd.read_csv(arquivo_csv, on_bad_lines='skip')
        df_temp.columns = df_temp.columns.str.strip()
        #df_temp = df_temp[['Name', 'DoB','Division','Based','Club']]
        lista_de_dataframes.append(df_temp)
    except ValueError as e:
        print(f"Aviso: Colunas 'Name' ou 'DoB' não encontradas em {arquivo_csv}. Pulando arquivo. Detalhe: {e}")
df = pd.concat(lista_de_dataframes, ignore_index=True)

display(df)

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,12/09/2020,12:30,Fulham,Arsenal,0,3,A,0,1,...,1.84,0.75,2.01,1.89,2.02,1.91,2.13,1.92,2.02,1.87
1,E0,12/09/2020,15:00,Crystal Palace,Southampton,1,0,H,1,0,...,1.70,0.25,1.78,2.13,1.79,2.17,1.85,2.18,1.79,2.12
2,E0,12/09/2020,17:30,Liverpool,Leeds,4,3,H,3,2,...,2.62,-1.50,1.85,2.05,1.85,2.08,1.90,2.16,1.84,2.04
3,E0,12/09/2020,20:00,West Ham,Newcastle,0,2,A,0,0,...,1.92,-0.50,2.03,1.87,2.04,1.88,2.09,1.91,2.02,1.86
4,E0,13/09/2020,14:00,West Brom,Leicester,0,3,A,0,0,...,1.73,0.25,1.92,1.98,1.93,1.99,1.95,2.01,1.91,1.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,E0,22/05/2022,16:00,Crystal Palace,Man United,1,0,H,1,0,...,2.04,0.25,1.68,2.15,1.74,2.23,1.88,2.25,1.74,2.16
756,E0,22/05/2022,16:00,Leicester,Southampton,4,1,H,0,0,...,2.63,-0.75,1.83,2.07,1.88,2.03,1.94,2.26,1.87,2.01
757,E0,22/05/2022,16:00,Liverpool,Wolves,3,1,H,1,1,...,3.28,-2.50,2.02,1.77,2.06,1.83,2.19,1.99,2.07,1.80
758,E0,22/05/2022,16:00,Man City,Aston Villa,3,2,H,0,1,...,3.36,-2.25,2.06,1.84,2.05,1.86,2.09,2.03,2.01,1.87


In [38]:
times = []
for time in df['HomeTeam'].unique():
    row = f"INSERT IGNORE INTO Time (c_nome_time, c_cidade_time, c_tecnico_time) VALUES ('{time}', NULL, NULL);"
    times.append(row)

times.append(f"INSERT IGNORE INTO Time (c_nome_time, c_cidade_time, c_tecnico_time) VALUES ('Wolves', NULL, NULL);")
# 4. Salvar o resultado em um arquivo de texto
DIR = 'inserts/txt_premier/insert_times_premier.txt'
try:
    with open(DIR, 'w', encoding='utf-8') as f:
        f.write('\n'.join(times))
    print(f"Arquivo '{DIR}' criado com sucesso!")
except Exception as e:
    print(f"Ocorreu um erro ao salvar o arquivo: {e}")

Arquivo 'inserts/txt_premier/insert_times_premier.txt' criado com sucesso!


In [39]:
import pandas as pd
import re

# --- Bloco de código inicial (mantido como estava) ---
colunas = ['Date','Time','FTAG','FTHG','HomeTeam','AwayTeam']
df_limpo = df[colunas].copy()
df_limpo = df_limpo.rename(columns={
    'FTAG': 'n_placar_visitante',
    'FTHG': 'n_placar_casa',
    'HomeTeam': 'c_time_casa',
    'AwayTeam': 'c_time_visitante'
})

# --- Lógica corrigida para data e temporada ---

# 1. Criar a coluna datetime completa e ordenar o DataFrame por ela.
#    Este passo é CRUCIAL para garantir que a contagem de jogos seja sequencial.
df_limpo['dt_data_horario'] = pd.to_datetime(df_limpo['Date'] + ' ' + df_limpo['Time'], format='%d/%m/%Y %H:%M')
df_limpo = df_limpo.sort_values(by='dt_data_horario').reset_index(drop=True)

# 2. Criar um identificador numérico para a temporada usando a sua lógica de 380 jogos.
#    A divisão inteira (//) agrupa os jogos de 380 em 380.
#    O resultado será 0 para os primeiros 380 jogos, 1 para os 380 seguintes, e assim por diante.
season_identifier = df_limpo.index // 380

# 3. Criar a coluna 'd_ano_campeonato' com o ano de início da temporada.
#    Como você disse que começa em 2019, somamos o identificador a 2019.
df_limpo['d_ano_campeonato'] = 2021 + season_identifier

# 4. Criar o nome do campeonato no formato "YYYY-YYYY".
#    Convertemos o ano para string e concatenamos com o ano seguinte.
df_limpo['c_nome_campeonato'] = "Premier League " + (df_limpo['d_ano_campeonato'] - 1).astype(str) + "-" + df_limpo['d_ano_campeonato'].astype(str)

# 5. Limpeza final: remover colunas originais de data/hora que não são mais necessárias.
df_limpo = df_limpo.drop(columns=['Date', 'Time'])

# Resetar o índice para começar em 1, como você tinha feito
df_limpo.index = range(1, len(df_limpo) + 1)

# Exibir o resultado final
display(df_limpo[['dt_data_horario', 'c_time_casa', 'c_time_visitante', 'd_ano_campeonato', 'c_nome_campeonato']].head())
display(df_limpo[['dt_data_horario', 'c_time_casa', 'c_time_visitante', 'd_ano_campeonato', 'c_nome_campeonato']].tail())

,dt_data_horario,c_time_casa,c_time_visitante,d_ano_campeonato,c_nome_campeonato
1,2020-09-12 12:30:00,Fulham,Arsenal,2021,Premier League 2020-2021
2,2020-09-12 15:00:00,Crystal Palace,Southampton,2021,Premier League 2020-2021
3,2020-09-12 17:30:00,Liverpool,Leeds,2021,Premier League 2020-2021
4,2020-09-12 20:00:00,West Ham,Newcastle,2021,Premier League 2020-2021
5,2020-09-13 14:00:00,West Brom,Leicester,2021,Premier League 2020-2021


,dt_data_horario,c_time_casa,c_time_visitante,d_ano_campeonato,c_nome_campeonato
756,2022-05-22 16:00:00,Brighton,West Ham,2022,Premier League 2021-2022
757,2022-05-22 16:00:00,Brentford,Leeds,2022,Premier League 2021-2022
758,2022-05-22 16:00:00,Arsenal,Everton,2022,Premier League 2021-2022
759,2022-05-22 16:00:00,Burnley,Newcastle,2022,Premier League 2021-2022
760,2022-05-22 16:00:00,Norwich,Tottenham,2022,Premier League 2021-2022


In [40]:
# Certifique-se que o DataFrame está ordenado por campeonato e data
# Se você já o ordenou antes, pode pular esta linha
df_limpo = df_limpo.sort_values(by=['c_nome_campeonato', 'dt_data_horario']).reset_index(drop=True)

# Cria uma contagem cumulativa dentro de cada grupo de campeonato,
# divide por 10 para agrupar os jogos em rodadas, e soma 1 para começar em 1.
df_limpo['n_rodada'] = (df_limpo.groupby('c_nome_campeonato').cumcount() // 10) + 1

# Exibe o DataFrame para verificar o resultado
display(df_limpo)

,n_placar_visitante,n_placar_casa,c_time_casa,c_time_visitante,dt_data_horario,d_ano_campeonato,c_nome_campeonato,n_rodada
0,3,0,Fulham,Arsenal,2020-09-12 12:30:00,2021,Premier League 2020-2021,1
1,0,1,Crystal Palace,Southampton,2020-09-12 15:00:00,2021,Premier League 2020-2021,1
2,3,4,Liverpool,Leeds,2020-09-12 17:30:00,2021,Premier League 2020-2021,1
3,2,0,West Ham,Newcastle,2020-09-12 20:00:00,2021,Premier League 2020-2021,1
4,3,0,West Brom,Leicester,2020-09-13 14:00:00,2021,Premier League 2020-2021,1
...,...,...,...,...,...,...,...,...
755,1,3,Brighton,West Ham,2022-05-22 16:00:00,2022,Premier League 2021-2022,38
756,2,1,Brentford,Leeds,2022-05-22 16:00:00,2022,Premier League 2021-2022,38
757,1,5,Arsenal,Everton,2022-05-22 16:00:00,2022,Premier League 2021-2022,38
758,2,1,Burnley,Newcastle,2022-05-22 16:00:00,2022,Premier League 2021-2022,38


In [41]:
df_limpo.to_csv("inserts/txt_premier/jogos_premier.csv", index=False)

In [42]:
# Conta a ocorrência de cada valor único na coluna 'c_nome_campeonato'
contagem_jogos = df_limpo['c_nome_campeonato'].value_counts()

# Imprime o resultado
print(contagem_jogos)

c_nome_campeonato
Premier League 2020-2021    380
Premier League 2021-2022    380
Name: count, dtype: int64


In [43]:
display(df_limpo)

,n_placar_visitante,n_placar_casa,c_time_casa,c_time_visitante,dt_data_horario,d_ano_campeonato,c_nome_campeonato,n_rodada
0,3,0,Fulham,Arsenal,2020-09-12 12:30:00,2021,Premier League 2020-2021,1
1,0,1,Crystal Palace,Southampton,2020-09-12 15:00:00,2021,Premier League 2020-2021,1
2,3,4,Liverpool,Leeds,2020-09-12 17:30:00,2021,Premier League 2020-2021,1
3,2,0,West Ham,Newcastle,2020-09-12 20:00:00,2021,Premier League 2020-2021,1
4,3,0,West Brom,Leicester,2020-09-13 14:00:00,2021,Premier League 2020-2021,1
...,...,...,...,...,...,...,...,...
755,1,3,Brighton,West Ham,2022-05-22 16:00:00,2022,Premier League 2021-2022,38
756,2,1,Brentford,Leeds,2022-05-22 16:00:00,2022,Premier League 2021-2022,38
757,1,5,Arsenal,Everton,2022-05-22 16:00:00,2022,Premier League 2021-2022,38
758,2,1,Burnley,Newcastle,2022-05-22 16:00:00,2022,Premier League 2021-2022,38


In [44]:
jogos = []
for index, row in df_limpo.iterrows():
    # Para cada linha, definimos as variáveis que precisam ser formatadas como texto para o SQL
    dt_data_horario = f"'{row['dt_data_horario'].strftime('%Y-%m-%d %H:%M:%S')}'"
    c_nome_campeonato = f"'{str(row['c_nome_campeonato']).replace("'", "''")}'"
    c_time_casa = f"'{str(row['c_time_casa']).replace("'", "''")}'"
    c_time_visitante = f"'{str(row['c_time_visitante']).replace("'", "''")}'"
    c_status = "'Agendado'"

    # A sua string original, agora usando as variáveis formatadas e os dados da linha ('row')
    query = ("INSERT IGNORE INTO Jogo "
           "(dt_data_horario, n_rodada, n_placar_casa, n_placar_visitante, c_nome_campeonato, "
           "d_ano_campeonato, c_nome_estadio, c_time_casa, c_time_visitante, c_status) "
           f"VALUES ({dt_data_horario}, {row['n_rodada']}, {row['n_placar_casa']}, "
           f"{row['n_placar_visitante']}, {c_nome_campeonato}, {row['d_ano_campeonato']}, "
           f"NULL, {c_time_casa}, {c_time_visitante}, {c_status});")
    jogos.append(query)

# 4. Salvar o resultado em um arquivo de texto
DIR = 'inserts/txt_premier/insert_jogos_premier.txt'
try:
    with open(DIR, 'w', encoding='utf-8') as f:
        f.write('\n'.join(jogos))
    print(f"Arquivo '{DIR}' criado com sucesso!")
except Exception as e:
    print(f"Ocorreu um erro ao salvar o arquivo: {e}")

Arquivo 'inserts/txt_premier/insert_jogos_premier.txt' criado com sucesso!


In [45]:
lista_de_inserts_times_campeonato = []
# Pega as combinações únicas de time, campeonato e ano.
# Usamos 'c_time_casa' como a fonte de todos os times.
times_por_campeonato = df_limpo[['c_time_casa', 'c_nome_campeonato', 'd_ano_campeonato']].drop_duplicates().sort_values(by=['d_ano_campeonato', 'c_time_casa'])
times_por_campeonato = times_por_campeonato.rename(columns={'c_time_casa': 'c_nome_time'})

for index, row in times_por_campeonato.iterrows():
    # Formata os valores de texto para SQL
    nome_time_sql = f"'{str(row['c_nome_time']).replace("'", "''")}'"
    nome_campeonato_sql = f"'{str(row['c_nome_campeonato']).replace("'", "''")}'"
    ano_campeonato = row['d_ano_campeonato']

    # Cria a query de INSERT
    query = f"INSERT IGNORE INTO Time_participa_campeonato (c_nome_time, c_nome_campeonato, d_ano_campeonato) VALUES ({nome_time_sql}, {nome_campeonato_sql}, {ano_campeonato});"
    lista_de_inserts_times_campeonato.append(query)

# 4. Salvar o resultado em um arquivo de texto
DIR = 'inserts/txt_premier/insert_t_c_premier.txt'
try:
    with open(DIR, 'w', encoding='utf-8') as f:
        f.write('\n'.join(lista_de_inserts_times_campeonato))
    print(f"Arquivo '{DIR}' criado com sucesso!")
except Exception as e:
    print(f"Ocorreu um erro ao salvar o arquivo: {e}")

Arquivo 'inserts/txt_premier/insert_t_c_premier.txt' criado com sucesso!
